In [ ]:
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd

load_dotenv()
api_key = os.getenv("google_api")
gmaps = googlemaps.Client(key=api_key)
GDRIVE_FOLDER = Path(os.getenv("GDRIVE_FOLDER"))
origins = GDRIVE_FOLDER / "origin.csv"
destinations = GDRIVE_FOLDER / "attractions.csv"
origin_df = pd.read_csv(origins)
destinations_df = pd.read_csv(destinations)
now = datetime.now()
mode = "transit"


origin = (
    39.946196,
    -75.139832,
)  # hardcoded here since my origin doesn't change, but if yours does you can tweak the function to accept origins.


def distance_duration_iteration():
    driving_list = []
    transit_list = []

    def distance_duration(destination):
        output = gmaps.directions(
            origin,
            destination,
            mode,
            units="imperial",
            departure_time=now,
        )

        if mode == "driving":
            driving_list.append(output)

        if mode == "transit":
            transit_list.append(output)

    """loops through the destinations file and runs the distance duration function"""
    for idx, row in destinations_df.iterrows():
        lat = row["Latitude"]
        lon = row["Longitude"]
        destination = (lat, lon)
        distance_duration(destination)

    if mode == "driving":
        print("returning driving list")
        return driving_list
    if mode == "transit":
        print("returning transit list")
        return transit_list


def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the google api returns"""
    labels = ["distance", "duration", "lat", "lng"]
    df = pd.DataFrame(columns=labels)

    try:
        for item in list_of_dicts:
            destination_lat = item[0]["legs"][0]["end_location"]["lat"]
            destination_lng = item[0]["legs"][0]["end_location"]["lng"]
            distance = item[0]["legs"][0]["distance"]["text"]
            duration = item[0]["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
    except:
        for item in list_of_dicts[0]:
            destination_lat = item[0]["legs"][0]["end_location"]["lat"]
            destination_lng = item[0]["legs"][0]["end_location"]["lng"]
            distance = item[0]["legs"][0]["distance"]["text"]
            duration = item[0]["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
    else:
        print("Something went wrong.")

    return df


def df_to_csv(df):
    df.to_csv(GDRIVE_FOLDER / f"{mode}_travel_times.csv", sep=",")


unpacked_transit = distance_duration_iteration()

In [18]:
def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the google api returns"""
    labels = ["distance", "duration", "lat", "lng"]
    df = pd.DataFrame(columns=labels)

    try:
        for item in list_of_dicts:
            destination_lat = item[0]["legs"][0]["end_location"]["lat"]
            destination_lng = item[0]["legs"][0]["end_location"]["lng"]
            distance = item[0]["legs"][0]["distance"]["text"]
            duration = item[0]["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
    except:
        for item in list_of_dicts[0]:
            print(item)
            destination_lat = item["legs"][0]["end_location"]["lat"]
            destination_lng = item["legs"][0]["end_location"]["lng"]
            distance = item["legs"][0]["distance"]["text"]
            duration = item["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
            print("transit looks good")
    else:
        print("Something went wrong.")

    return df

unpack_dicts(unpacked_transit)

def df_to_csv(df):
    df.to_csv(GDRIVE_FOLDER / f"{mode}_travel_times.csv", sep=",")


{'bounds': {'northeast': {'lat': 39.9462187, 'lng': -75.1179899}, 'southwest': {'lat': 39.7341109, 'lng': -75.31965819999999}}, 'copyrights': 'Map data ©2022 Google', 'legs': [{'arrival_time': {'text': '7:22pm', 'time_zone': 'America/New_York', 'value': 1646094147}, 'departure_time': {'text': '5:43pm', 'time_zone': 'America/New_York', 'value': 1646088238}, 'distance': {'text': '21.2 mi', 'value': 34187}, 'duration': {'text': '1 hour 38 mins', 'value': 5909}, 'end_address': '95 Woodstown Rd, Swedesboro, NJ 08085, USA', 'end_location': {'lat': 39.7343775, 'lng': -75.3191161}, 'start_address': '211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA', 'start_location': {'lat': 39.9462187, 'lng': -75.1398433}, 'steps': [{'distance': {'text': '0.7 mi', 'value': 1073}, 'duration': {'text': '6 mins', 'value': 362}, 'end_location': {'lat': 39.9445308, 'lng': -75.1301929}, 'html_instructions': 'Walk to Camden Adventure Aquarium', 'polyline': {'points': '{~xrF~vriMDUZ_D|@_JDWFYDSFUFQFQdBwCh